In [1]:
import numpy as np

from itertools import product

In [2]:
# constructing the phase augmented w state

num_qubits = 10

state_dim = 1 << num_qubits                     # bit shifting a ...0001 bitstring is the same as 2**a
w_aug = np.zeros(state_dim, dtype=complex)      # empty state vector

# since the W state has only non-zero amplitudes for one-hot states, we only need num_qubits random phases
rng = np.random.default_rng(42)
thetas = rng.uniform(0, 2*np.pi, size=num_qubits)

for j in range(num_qubits):
    idx = 1 << (num_qubits - 1 - j)              # find indexing mask via bit shifting

    # apply the phase to the corresponding amplitude coefficient
    w_aug[idx] = np.exp(1j * thetas[j]) / np.sqrt(num_qubits)


def format_bytes(b):
    for u in ['B', 'KB', 'MB', 'GB', 'TB']:
        if b < 1024:
            return f"{b:.2f} {u}"
        b /= 1024


print(f"Size of state vector in memory: {format_bytes(w_aug.nbytes)} \n")

for i in range(10):
    print(f"{i:0{num_qubits}b}: {w_aug[i]:.2f} + {w_aug[i].imag:.2f}j")

Size of state vector in memory: 16.00 KB 

0000000000: 0.00+0.00j + 0.00j
0000000001: -0.30+0.10j + 0.10j
0000000010: 0.22+0.23j + 0.23j
0000000011: 0.00+0.00j + 0.00j
0000000100: 0.07-0.31j + -0.31j
0000000101: 0.00+0.00j + 0.00j
0000000110: 0.00+0.00j + 0.00j
0000000111: 0.00+0.00j + 0.00j
0000001000: 0.02-0.32j + -0.32j
0000001001: 0.00+0.00j + 0.00j


In [3]:
class PauliOperator:
    def __init__(self, matrix, eigenvalues, eigenvectors):
        """
        Bundles the relevant information of a Pauli operator into a single object.

        REMARK: The eigenvalues and eigenvectors are directly passed to avoid numerical inaccuracies.

        :param matrix: The matrix representation of the Pauli operator.
        :param eigenvalues: The eigenvalues of the Pauli operator.
        :param eigenvectors: The eigenvectors of the Pauli operator.
        """
        self.matrix = matrix
        self.eigenvalues = eigenvalues
        self.eigenvectors = eigenvectors


class MultiQubitMeasurementSetting:
    def __init__(self, measurement_settings):
        """
        Represents a multi-qubit measurement setting using Pauli operators.

        :param measurement_settings: A list of Pauli operators representing the measurement setting for each qubit.
        """
        self._measurement_settings = measurement_settings
        self.eigenvectors = []
        self.eigenvalues = []

        self._construct_basis()

    def _construct_basis(self):
        """
        Constructs the multi-qubit basis vectors and their corresponding eigenvalues from the given Pauli operators.
        """

        # in this first loop we iterate over the computational basis states of the n-qubit system
        # constructing them as a list of 0s and 1s allows us to leverage pythons addressing capabilities
        for state_addressing in product(range(2), repeat=len(self._measurement_settings)):
            multi_qubit_eigenvalue = 1.0
            multi_qubit_eigenvector = None

            # we pair up the Pauli operator for each single qubit basis state and retrieve the matching eigenvalue and eigenvector
            for pauli_op, state_index in zip(self._measurement_settings, state_addressing):
                eigenvalue = pauli_op.eigenvalues[state_index]
                eigenvector = pauli_op.eigenvectors[state_index]

                # the eigenvalue of the multi-qubit operator is the product of the single-qubit eigenvalues
                multi_qubit_eigenvalue *= eigenvalue

                # we build up the multi-qubit eigenvector by taking the tensor product of the single-qubit eigenvectors
                if multi_qubit_eigenvector is None:
                    multi_qubit_eigenvector = eigenvector
                else:
                    multi_qubit_eigenvector = np.kron(multi_qubit_eigenvector, eigenvector)

            self.eigenvalues.append(multi_qubit_eigenvalue)
            self.eigenvectors.append(multi_qubit_eigenvector)

In [4]:
sigma_0 = PauliOperator(np.eye(2), [1, 1], [np.array([1, 0]), np.array([0, 1])])
sigma_x = PauliOperator(np.array([[0, 1], [1, 0]]), [1, -1], [np.array([1, 1]) / np.sqrt(2), np.array([1, -1]) / np.sqrt(2)])
sigma_y = PauliOperator(np.array([[0, -1j], [1j, 0]]), [1, -1], [np.array([1, -1j]) / np.sqrt(2), np.array([1, 1j]) / np.sqrt(2)])
sigma_z = PauliOperator(np.array([[1, 0], [0, -1]]), [1, -1], [np.array([1, 0]), np.array([0, 1])])
pauli_operators = { 'I': sigma_0, 'X': sigma_x, 'Y': sigma_y, 'Z': sigma_z }

In [5]:
def sample_state(state, pauli_labels, num_samples=1000):
    """
    Draw samples from |state⟩ in the tensor-product basis defined by pauli_labels
    and return **measurement bit-strings**.

    Convention
    ───────────
    * For each qubit the character is
      * '0'  → eigenvalue +1 of the chosen Pauli operator
      * '1'  → eigenvalue −1
    * The i-th character in the string refers to pauli_labels[i].

    Example
    --------
    >>> measurement_directions = ['Z', 'X', 'I', 'Y']
    >>> sample_state(w_aug, measurement_directions, num_samples=1)
    '0110'
    """
    # 1. Build the multi-qubit measurement setting
    measurement_settings = [pauli_operators[pl] for pl in pauli_labels]
    meas_setting = MultiQubitMeasurementSetting(measurement_settings)

    # 2. Born-rule probabilities for the eigenbasis {|φ_k⟩}
    probs = np.array([np.abs(np.vdot(vec, state))**2 for vec in meas_setting.eigenvectors])
    probs /= probs.sum()                     # numerical hygiene

    # 3. Draw raw indices of eigenvectors
    raw_indices = np.random.choice(len(probs), size=num_samples, p=probs)

    # 4. Convert index → bit-string (lexicographic order of itertools.product
    #    already matches binary counting with the leftmost qubit as MSB)
    def idx_to_bitstring(idx: int) -> str:
        return format(idx, f'0{len(pauli_labels)}b')

    bitstrings = [idx_to_bitstring(i) for i in raw_indices]
    return bitstrings[0] if num_samples == 1 else bitstrings


In [6]:
%time

measurement_directions = rng.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)
outcome = sample_state(w_aug, measurement_directions, num_samples=1000)
print(measurement_directions)  # e.g. ['Y' 'X' 'Z' 'I']
print(outcome)                 # e.g. '1010'

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs
['Y' 'X' 'I' 'Z' 'Z' 'Y' 'X' 'Z' 'Y' 'X']
['0100000010', '0100000011', '1010011000', '0001000000', '1100010011', '1000101010', '1000001001', '1000011010', '0100000011', '1000000001', '1000000100', '1000010001', '1100000011', '0010011011', '1100000110', '1010001010', '0000010011', '1000101000', '0100010011', '1100001000', '1000010011', '1110011000', '0100010011', '0000010000', '0101000001', '1110011010', '0100000110', '1100010110', '1000001001', '0000001000', '1100100010', '0000011101', '1100000010', '0100001110', '1100000011', '1100110000', '1000001000', '0001000011', '1000001010', '0000000110', '0100001010', '0100001011', '0100001001', '1000000101', '1000001001', '1000100000', '0100011011', '1100101001', '1000011000', '1000001010', '0101001000', '0000010100', '0001011011', '1101010011', '0000001010', '0100010011', '0100010001', '1010010011', '1110010011', '0100010101', '1000000100', '0000111011', '0100111011', '000

In [ ]:
# crosschecking the statistics with the original state amplitudes
import

# average per outcome
outcome_counts = Counter(outcome)
outcome_probs = np.array([outcome_counts[bs] for bs in outcome_counts]) / len(outcome)
print(outcome_probs)
